### Import libraries and define constants

In [ ]:
import matplotlib
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#import data_describe as mw
matplotlib.style.use('ggplot')
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 100)
pd.set_option('max_columns', None)
warnings.filterwarnings('ignore')

In [ ]:
import os
from datetime import datetime
from google.cloud import bigquery
import pandas as pd
from collections import Counter
from google.cloud import aiplatform
from google.cloud.aiplatform import Feature, Featurestore
from google.cloud.aiplatform_v1.types import featurestore_service

In [ ]:
from google.cloud.aiplatform_v1 import (FeaturestoreOnlineServingServiceClient,
                                        FeaturestoreServiceClient)

from google.cloud.aiplatform_v1.types import FeatureSelector, IdMatcher
from google.cloud.aiplatform_v1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1.types import feature as feature_pb2
from google.cloud.aiplatform_v1.types import featurestore as featurestore_pb2

from google.cloud.aiplatform_v1.types import featurestore_online_service as featurestore_online_service_pb2

from google.cloud.aiplatform_v1.types import featurestore_service as featurestore_service_pb2

from google.cloud.aiplatform_v1.types import io as io_pb2
from google.protobuf.duration_pb2 import Duration

In [ ]:
PROJECT_ID = "ds-training-380514"
REGION = "us-central1"  # @param {type:"string"}
ONLINE_STORE_FIXED_NODE_COUNT=1
    
print(f'Project ID: {PROJECT_ID}\nRegion : {REGION}')

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
bucket = "chicago-taxi-ay"
!gsutil mb -c standard -l {REGION} gs://{bucket}

In [ ]:
API_ENDPOINT = "us-central1-aiplatform.googleapis.com"  # @param {type:"string"}

In [ ]:
# admin_client for CRUD 
admin_client = FeaturestoreServiceClient(client_options={"api_endpoint": API_ENDPOINT})

# data_client for reading feature values
data_client = FeaturestoreOnlineServingServiceClient(
                                                     client_options={"api_endpoint": API_ENDPOINT}
                                                    )

In [ ]:
# resource path of feature-store
BASE_RESOURCE_PATH = admin_client.common_location_path(PROJECT_ID, REGION)

## Create FeatureStore
The method to create a Featurestore returns a
[long-running operation](https://google.aip.dev/151) (LRO). An LRO starts an asynchronous job. LROs are returned for other API
methods too, such as updating or deleting a featurestore. Running the code cell will create a featurestore and print the process log.

a value of zero for ONLINE_STORE_FIXED_NODE_COUNT mean no online feature store 

In [ ]:
FEATURESTORE_ID = "teststore3"

try:
    # long-running-operation; 
    create_lro = admin_client.create_featurestore(
                    featurestore_service_pb2.CreateFeaturestoreRequest(
                        parent=BASE_RESOURCE_PATH,
                        featurestore_id=FEATURESTORE_ID,
                        featurestore=featurestore_pb2.Featurestore(
                            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                                fixed_node_count=ONLINE_STORE_FIXED_NODE_COUNT
                            ),
                        ),
                    )
                )
    # Wait for LRO to finish and get the LRO result.
    print(create_lro.result())
except Exception as e:
    print(e)

### Alternative method of creating Feature store 

fs = Featurestore.create(
    featurestore_id=FEATURESTORE_ID,
    online_store_fixed_node_count=ONLINE_STORE_FIXED_NODE_COUNT,
    project=PROJECT_ID,
    location=REGION,
    sync=True,
)

#### Get details of the Feature store

In [ ]:
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=PROJECT_ID,
    location=REGION,
)
print(fs.gca_resource)

## Create Entity-Type
Entity types can be created within the Featurestore class. Below, create the Users entity type and Movies entity type. A process log will be printed out.

Entity-Type 1: taxis

In [ ]:
try:
    users_entity_type_lro = admin_client.create_entity_type(
                                featurestore_service_pb2.CreateEntityTypeRequest(
                                    parent=admin_client.featurestore_path(PROJECT_ID,
                                                                          REGION,
                                                                          FEATURESTORE_ID),
                                    entity_type_id="taxis",
                                    entity_type=entity_type_pb2.EntityType(
                                                        description="taxi details",
                                                                          ),
                                                                                )
                                                            )
    # wait for EntityType creation operation
    print(users_entity_type_lro.result())
except Exception as e:
    print(e)

Entity-Type 2: companies

In [ ]:
try:
    products_entity_type_lro = admin_client.create_entity_type(
                                    featurestore_service_pb2.CreateEntityTypeRequest(
                                        parent=admin_client.featurestore_path(PROJECT_ID,
                                                                              REGION,
                                                                              FEATURESTORE_ID),
                                        entity_type_id="companies",
                                        entity_type=entity_type_pb2.EntityType(description="company details"),
                                                                                    )
                                                              )

    # wait for EntityType creation operation
    print(products_entity_type_lro.result())
except Exception as e:
    print(e)

In [ ]:
### Alterative function to create entity
# Create users entity type
users_entity_type = fs.create_entity_type(
    entity_type_id="taxis",
    description="taxi details",
)

To retrieve an entity type or check that it has been created use the [get_entity_type](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L106) or [list_entity_types](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L278) methods on the Featurestore object.

In [ ]:
taxis_entity_type = fs.get_entity_type(entity_type_id="taxis")
companies_entity_type = fs.get_entity_type(entity_type_id="companies")
print(taxis_entity_type)
print(companies_entity_type)

In [ ]:
fs.list_entity_types()

## Enable monitoring on EntityType e.g. train-serving skew

In [ ]:
from google.cloud.aiplatform_v1beta1 import \
    FeaturestoreServiceClient as v1beta1_FeaturestoreServiceClient

from google.cloud.aiplatform_v1beta1.types import \
    entity_type as v1beta1_entity_type_pb2

from google.cloud.aiplatform_v1beta1.types import \
    featurestore_monitoring as v1beta1_featurestore_monitoring_pb2

from google.cloud.aiplatform_v1beta1.types import \
    featurestore_service as v1beta1_featurestore_service_pb2

In [ ]:
v1beta1_admin_client = v1beta1_FeaturestoreServiceClient(
                                                         client_options={"api_endpoint": API_ENDPOINT}
                                                        )

In [ ]:
# Enable monitoring
# All Features belonging to this Entity-Type will, by default, inherit the monitoring config.

v1beta1_admin_client.update_entity_type(
    v1beta1_featurestore_service_pb2.UpdateEntityTypeRequest(
        entity_type=v1beta1_entity_type_pb2.EntityType(
                        name=admin_client.entity_type_path(
                                                           PROJECT_ID,
                                                           REGION,
                                                           FEATURESTORE_ID,
                                                           "taxis"
                                                          ),
                        monitoring_config=v1beta1_featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                            snapshot_analysis=v1beta1_featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                                                        monitoring_interval=Duration(seconds=86400),  # 1 day
                                                                                                                               ),
                                                                                                          ),
                                                    ),
                                                           )
                                       )

In [ ]:
# Enable monitoring
# All Features belonging to this Entity-Type will, by default, inherit the monitoring config.

v1beta1_admin_client.update_entity_type(
    v1beta1_featurestore_service_pb2.UpdateEntityTypeRequest(
        entity_type=v1beta1_entity_type_pb2.EntityType(
                        name=admin_client.entity_type_path(
                                                           PROJECT_ID,
                                                           REGION,
                                                           FEATURESTORE_ID,
                                                           "companies"
                                                          ),
                        monitoring_config=v1beta1_featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                            snapshot_analysis=v1beta1_featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                                                        monitoring_interval=Duration(seconds=86400),  # 1 day
                                                        staleness_days=1,
                                                                                                                               ),
                                                                                                          ),
                                                    ),
                                                           )
                                       )

## Create Features

**Note**: feature_id must begin with a lowercase or underscore and can only contain lowercase, numbers, and underscores.
Features can be created within each entity type. Add defining features to the Users entity type and Movies entity type by using the following methods.

For Entity-Type 1 i.e. 'taxis'

In [ ]:
try:
    admin_client.batch_create_features(
        parent=admin_client.entity_type_path(
                                             PROJECT_ID, 
                                             REGION, 
                                             FEATURESTORE_ID, 
                                             "taxis"
                                            ),
        requests=[
            # 1 for each column
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.INT64,
                                                    description="10 days rolling trip count of individual taxi",
                                                                           ),
                                                feature_id="_10d_rolling_trip_count",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling average fare of individual taxi",
                                                                           ),
                                                feature_id="_10d_rolling_avg_fare",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling average tips of individual taxi",
                                                                           ),
                                                feature_id="_10d_rolling_avg_tips",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.INT64,
                                                    description="10 days rolling trip with tips count of individual taxi",
                                                                           ),
                                                feature_id="_10d_rolling_tips_count",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling average miles of individual taxi",
                                                                           ),
                                                feature_id="_10d_rolling_avg_miles",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling average trip duration in second of individual taxi",
                                                                           ),
                                                feature_id="_10d_rolling_avg_seconds",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling probability of trip with tips of individual taxi",
                                                                           ),
                                                feature_id="_10d_rolling_tips_proba",
                                                         ),
                ],
    ).result()
except Exception as e:
    print(e)

Adding target/label column

For Entity-Type 2 i.e. 'companies'

In [ ]:
try:
    admin_client.batch_create_features(
        parent=admin_client.entity_type_path(
                                             PROJECT_ID, 
                                             REGION, 
                                             FEATURESTORE_ID, 
                                             "companies"
                                            ),
        requests=[
            # 1 for each column
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.INT64,
                                                    description="10 days rolling trip count of all taxis under company",
                                                                           ),
                                                feature_id="_10d_rolling_comp_trip_count",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling average fare of all taxis under company",
                                                                           ),
                                                feature_id="_10d_rolling_comp_avg_fare",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling average tips of all taxis under company",
                                                                           ),
                                                feature_id="_10d_rolling_comp_avg_tips",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.INT64,
                                                    description="10 days rolling trip with tips count of all taxis under company",
                                                                           ),
                                                feature_id="_10d_rolling_comp_tips_count",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling average miles of all taxis under company",
                                                                           ),
                                                feature_id="_10d_rolling_comp_avg_miles",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling average trip duration in second of all taxis under company",
                                                                           ),
                                                feature_id="_10d_rolling_comp_avg_seconds",
                                                         ),
            featurestore_service_pb2.CreateFeatureRequest(
                                                feature=feature_pb2.Feature(
                                                    value_type=feature_pb2.Feature.ValueType.DOUBLE,
                                                    description="10 days rolling probability of trip with tips of all taxis under company",
                                                                           ),
                                                feature_id="_10d_rolling_comp_tips_proba",
                                                         ),
                ],
    ).result()
except Exception as e:
    print(e)

## Feature Transformation

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

In [ ]:
sql = """
SELECT 
    *
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` where trip_end_timestamp >= '2023-01-01'
"""
df = client.query(sql).to_dataframe()

In [ ]:
df_all = df.copy()
print(df.shape)
df.head()

In [ ]:
df = df[df['fare']>0]
df['tips_percent']=df.apply(lambda x: (x['tips']*100)/x['fare'], axis=1)
df['tips_flag'] = df['tips'].transform(lambda x: 1 if x>0 else 0)
df['date'] = df['trip_start_timestamp'].transform(lambda x: x.date())
df['date'] = df.date.astype('datetime64')
print(df.shape)
df.head(2)

In [ ]:
print(df.shape)

In [ ]:
df_all2 = df.copy()
print(df_all2.shape)
df_all2.head()

In [ ]:
company_df = df.copy()
company_df = company_df.groupby(['company','date']).agg({'fare':['count','mean'],'tips':'mean','tips_flag':'sum',
                                                   'trip_miles':'mean','trip_seconds':'mean'}).reset_index()
company_df.columns = company_df.columns.map(''.join)
company_df = company_df.rename(columns={'farecount':'comp_trip_count','faremean':'comp_avg_fare','tipsmean':'comp_avg_tips',
                                        'tips_flagsum':'comp_tips_count','trip_milesmean':'comp_avg_miles','trip_secondsmean':'comp_avg_seconds'})

company_df = company_df.groupby('company').rolling('10D', on='date').agg({'comp_trip_count':'sum','comp_avg_fare':'mean','comp_avg_tips':'mean',
                                                          'comp_tips_count':'sum','comp_avg_miles':'mean','comp_avg_seconds':'mean'}).reset_index()
company_df.columns = ['_10d_rolling_'+i if i not in ['company','date'] else i for i in company_df.columns]
company_df['_10d_rolling_comp_tips_proba'] = company_df.apply(lambda x: x['_10d_rolling_comp_tips_count']/x['_10d_rolling_comp_trip_count'] 
                                                  if x['_10d_rolling_comp_trip_count']!=0 else 0, axis=1)
company_df['update_timestamp'] = company_df['date'].apply(pd.to_datetime, utc=True)
company_df['update_timestamp'] = company_df['update_timestamp'].apply(lambda x: x.isoformat())
company_df = company_df.drop(columns=['date'])
company_df['_10d_rolling_comp_trip_count'] = company_df['_10d_rolling_comp_trip_count'].astype('int')
company_df['_10d_rolling_comp_tips_count'] = company_df['_10d_rolling_comp_tips_count'].astype('int')
company_df.head()

In [ ]:
taxi_df = df.copy()
taxi_df = taxi_df.fillna(0).groupby(['taxi_id','date']).agg({'fare':['count','mean'],'tips':'mean','tips_flag':'sum',
                                                   'trip_miles':'mean','trip_seconds':'mean'}).reset_index()
taxi_df.columns = taxi_df.columns.map(''.join)
taxi_df = taxi_df.rename(columns={'farecount':'trip_count','faremean':'avg_fare','tipsmean':'avg_tips','tips_flagsum':'tips_count',
                                  'trip_milesmean':'avg_miles','trip_secondsmean':'avg_seconds'})

taxi_df = taxi_df.groupby('taxi_id').rolling('10D', on='date').agg({'trip_count':'sum','avg_fare':'mean','avg_tips':'mean',
                                                          'tips_count':'sum','avg_miles':'mean','avg_seconds':'mean'}).reset_index()
taxi_df.columns = ['_10d_rolling_'+i if i not in ['taxi_id','date'] else i for i in taxi_df.columns]
taxi_df['_10d_rolling_tips_proba'] = taxi_df.apply(lambda x: x['_10d_rolling_tips_count']/x['_10d_rolling_trip_count'] 
                                                   if x['_10d_rolling_trip_count']!=0 else 0, axis=1)
taxi_df['update_timestamp'] = taxi_df['date'].apply(pd.to_datetime, utc=True)
taxi_df['update_timestamp'] = taxi_df['update_timestamp'].apply(lambda x: x.isoformat())
taxi_df = taxi_df.drop(columns=['date'])
taxi_df['_10d_rolling_trip_count'] = taxi_df['_10d_rolling_trip_count'].astype('int')
taxi_df['_10d_rolling_tips_count'] = taxi_df['_10d_rolling_tips_count'].astype('int')
taxi_df.head()

In [ ]:
taxi_df_all = taxi_df.copy()
print(taxi_df.shape)
company_df_all = company_df.copy()
company_df.shape

In [ ]:
taxi_df = taxi_df[:200]
company_df = company_df[:200]

In [ ]:
print(f'taxi_df shape: {taxi_df.shape}\ncompany_df shape: {company_df.shape}')

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
taxi_df_csv_filename = f'transformed_taxi_features_{TIMESTAMP}.csv'
company_df_csv_filename = f'transformed_company_features_{TIMESTAMP}.csv'

taxi_df.to_csv(f'gs://{bucket}/feature_store/transformed/{taxi_df_csv_filename}', index=False)
company_df.to_csv(f'gs://{bucket}/feature_store/transformed/{company_df_csv_filename}', index=False)

## Import Feature Values
You need to import feature values before you can use them for online/offline serving. We can  import feature values from GCS BigQuery or a Pandas dataframe.
When importing, specify the following in your request:

*   IDs of the features to import 
*   Data source URI
*   Data source format: BigQuery Table/Avro/CSV
No matter what format you are using, each imported entity *must* have an ID; also, each entity can *optionally* have a timestamp, specifying when the feature values are generated. 

In [ ]:
def protobuf_timestamp(input_timestamp: str):
    import pandas as pd
    from google.protobuf.timestamp_pb2 import Timestamp
    from datetime import datetime
    from dateutil.parser import parse
    
    output_timestamp = datetime.timestamp(parse(input_timestamp))
    seconds = int(output_timestamp)
    millis  = int(round((output_timestamp - seconds),3) * 10**9)
    output_timestamp = Timestamp(seconds=seconds, nanos=millis)
    return output_timestamp

In [ ]:
df = pd.read_csv(f'gs://{bucket}/feature_store/transformed/{company_df_csv_filename}')
df.head()

In [ ]:
pb_timestamp = protobuf_timestamp(df['update_timestamp'][0])

Entity-Type 1 'taxis'

In [ ]:
import_users_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=admin_client.entity_type_path(
                                              PROJECT_ID,
                                              REGION,
                                              FEATURESTORE_ID,
                                              "taxis"
                                             ),
    csv_source=io_pb2.CsvSource(
                                  # Source
                                  gcs_source=io_pb2.GcsSource(
                                    uris=[
                                          f'gs://{bucket}/feature_store/transformed/{taxi_df_csv_filename}'
                                         ]
                                                             )
                                 ),
    entity_id_field="taxi_id",
    feature_specs=[
                    # Features
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_trip_count"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_avg_fare"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_avg_tips"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_tips_count"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_avg_miles"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_avg_seconds"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_tips_proba"),
                  ],
    feature_time_field='update_timestamp',
    worker_count=1,
)

In [ ]:
# Start to import
ingestion_lro = admin_client.import_feature_values(import_users_request)

In [ ]:
# Polls for the LRO status and prints when the LRO has completed
ingestion_lro.result()

**Note**: UI console is showing this ingestion job as success.

Entity-Type 2 'companies'

In [ ]:
import_users_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=admin_client.entity_type_path(
                                              PROJECT_ID,
                                              REGION,
                                              FEATURESTORE_ID,
                                              "companies"
                                             ),
    csv_source=io_pb2.CsvSource(
                                  # Source
                                  gcs_source=io_pb2.GcsSource(
                                    uris=[
                                          f'gs://{bucket}/feature_store/transformed/{company_df_csv_filename}'
                                         ]
                                                             )
                                 ),
    entity_id_field="company",
    feature_specs=[
                    # Features
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_comp_trip_count"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_comp_avg_fare"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_comp_avg_tips"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_comp_tips_count"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_comp_avg_miles"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_comp_avg_seconds"),
                    featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="_10d_rolling_comp_tips_proba"),
                  ],
    #feature_time_field='update_timestamp',
    feature_time=pb_timestamp,
    worker_count=1,
)

In [ ]:
# Start to import
ingestion_lro = admin_client.import_feature_values(import_users_request)

In [ ]:
# Polls for the LRO status and prints when the LRO has completed
ingestion_lro.result()

In [ ]:
taxi_df['taxi_id'].values[0]

## Online Serving
[Online serving](https://cloud.google.com/vertex-ai/docs/featurestore/serving-online)
lets you serve feature values for small batches of entities. It's designed for latency-sensitive service, such as online model prediction.

In [ ]:
# Fetch the following features
feature_selector = FeatureSelector(
    id_matcher=IdMatcher(ids=['_10d_rolling_trip_count', '_10d_rolling_avg_fare', '_10d_rolling_avg_tips', '_10d_rolling_tips_count', 
                              '_10d_rolling_avg_miles', '_10d_rolling_avg_seconds', '_10d_rolling_tips_proba',  'update_timestamp']
                        )
)
data_client.read_feature_values(
    featurestore_online_service_pb2.ReadFeatureValuesRequest(
        # Fetch from the following feature store/entity type
        entity_type=admin_client.entity_type_path(
            PROJECT_ID,
            REGION,
            FEATURESTORE_ID,
            'taxis'
        ),
        # Fetch the user features having a specific ID
        entity_id=taxi_df['taxi_id'].values[0],
        feature_selector=feature_selector,
    )
)

### Read one entity per request

With the Python SDK, it is easy to read feature values of one entity. By default, the SDK will return the  latest value of each feature, meaning the feature values with the most recent  timestamp.

To read feature values, specify the entity type ID and features to read. By default all the features of an entity type will be selected. The response will output and display the selected entity type ID and the selected feature values as a Pandas dataframe.

In [ ]:
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=PROJECT_ID,
    location=REGION,
)
print(fs.gca_resource)

In [ ]:
taxis_entity_type = fs.get_entity_type(entity_type_id="taxis")
companies_entity_type = fs.get_entity_type(entity_type_id="companies")
print(taxis_entity_type)
print(companies_entity_type)

In [ ]:
taxis_entity_type.read(entity_ids=taxi_df['taxi_id'].values[0])

### Read multiple entities per request

To read feature values from multiple entities, specify the different entity type IDs. By default all the features of an entity type will be selected. Note that fetching only a small number of entities is recommended when using this SDK due to its latency-sensitive nature.

In [ ]:
taxis_entity_type.read(entity_ids=[taxi_df['taxi_id'].values[0], taxi_df['taxi_id'].values[1]])

In [ ]:
# Fetch the following features
feature_selector = FeatureSelector(
    id_matcher=IdMatcher(ids=['_10d_rolling_comp_trip_count', '_10d_rolling_comp_avg_fare', '_10d_rolling_comp_avg_tips',
                              '_10d_rolling_comp_tips_count', '_10d_rolling_comp_avg_miles', '_10d_rolling_comp_avg_seconds',
                              '_10d_rolling_comp_tips_proba', 'update_timestamp']
                        )
)
data_client.read_feature_values(
    featurestore_online_service_pb2.ReadFeatureValuesRequest(
        # Fetch from the following feature store/entity type
        entity_type=admin_client.entity_type_path(
            PROJECT_ID,
            REGION,
            FEATURESTORE_ID,
            'companies'
        ),
        # Fetch the user features having a specific ID
        entity_id='24 Seven Taxi',
        feature_selector=feature_selector,
    )
)

## Create Batch Request CSV File

In [ ]:
INPUT_CSV_FILE = f'gs://{bucket}/transformed/feature_store/batch_request.csv'

In [ ]:
df_all2.head()

In [ ]:
batch_df = df_all2[df_all2['date']<'2023-05-23']

In [ ]:
batch_df = batch_df.groupby(['taxi_id','company'])['date'].max().reset_index()

In [ ]:
batch_df['timestamp'] = batch_df['date'].apply(pd.to_datetime, utc=True)
batch_df['timestamp'] = batch_df['timestamp'].apply(lambda x: x.isoformat())
batch_df = batch_df.drop(columns=['date'])
batch_df = batch_df.rename(columns={'taxi_id':'taxis','company':'companies'})

In [ ]:
batch_df.head()

In [ ]:
batch_df.to_csv(INPUT_CSV_FILE, index=False)

## Create BQ dataset for output

In [ ]:
# Output dataset
DESTINATION_DATA_SET = "chicago_taxi"  # @param {type:"string"}

# Output table
# Make sure that the table does NOT already exists
# Reason: BatchReadFeatureValues API cannot overwrite an existing table
DESTINATION_TABLE_NAME = "batch_request"  # @param {type:"string"}
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DESTINATION_TABLE_NAME = "{prefix}_{timestamp}".format(
                                                     prefix=DESTINATION_TABLE_NAME,
                                                     timestamp=TIMESTAMP
                                                    )

DESTINATION_PATTERN = "bq://{project}.{dataset}.{table}"
DESTINATION_TABLE_URI = DESTINATION_PATTERN.format(
                                                   project=PROJECT_ID,
                                                   dataset=DESTINATION_DATA_SET,
                                                   table=DESTINATION_TABLE_NAME
                                                  )

In [ ]:
try:
    # Create dataset
    client = bigquery.Client(project=PROJECT_ID)
    dataset_id = "{}.{}".format(client.project, DESTINATION_DATA_SET)
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = REGION
    dataset = client.create_dataset(dataset)

    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
except Exception as e:
    print(e)

## Batch Serving

In [ ]:
batch_serving_request = featurestore_service_pb2.BatchReadFeatureValuesRequest(
    # featurestore info
    featurestore=admin_client.featurestore_path(PROJECT_ID,
                                                REGION,
                                                FEATURESTORE_ID),
    # URL for the label data i.e. Table 1
    csv_read_instances=io_pb2.CsvSource(gcs_source=io_pb2.GcsSource(uris=[INPUT_CSV_FILE])),
    destination=featurestore_service_pb2.FeatureValueDestination(
        bigquery_destination=io_pb2.BigQueryDestination(
            # Output to BigQuery table created earlier
            output_uri=DESTINATION_TABLE_URI
        )
    ),
    entity_type_specs=[
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # Read the specific features from the 'users' entity-type
            entity_type_id="taxis",
            feature_selector=FeatureSelector(
                id_matcher=IdMatcher(ids=[
                    # features, use "*" if you want to select all features
                    '_10d_rolling_trip_count', '_10d_rolling_avg_fare', '_10d_rolling_avg_tips', '_10d_rolling_tips_count',
                    '_10d_rolling_avg_miles', '_10d_rolling_avg_seconds', '_10d_rolling_tips_proba']
                                    )
            ),
        ),
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # Read the specific feature values of the 'products' entity-type
            entity_type_id="companies",
            feature_selector=FeatureSelector(
                id_matcher=IdMatcher(ids=[
                    '_10d_rolling_comp_trip_count', '_10d_rolling_comp_avg_fare', '_10d_rolling_comp_avg_tips',
                    '_10d_rolling_comp_tips_count', '_10d_rolling_comp_avg_miles', '_10d_rolling_comp_avg_seconds',
                    '_10d_rolling_comp_tips_proba'])
            ),
        ),
    ],
)

In [ ]:
# Execute the batch read
batch_serving_lro = admin_client.batch_read_feature_values(batch_serving_request)

In [ ]:
# This long runing operation will poll until the batch read finishes.
batch_serving_lro.result()

### List of Feature store functions

In [ ]:
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=PROJECT_ID,
    location=REGION,
)
print(fs.gca_resource)

In [ ]:
fs.list_entity_types()

In [ ]:
taxis_entity_type = fs.get_entity_type(entity_type_id="taxis")
companies_entity_type = fs.get_entity_type(entity_type_id="companies")
print(taxis_entity_type)
print(companies_entity_type)

In [ ]:
taxis_entity_type.list_features()

In [ ]:
companies_entity_type.list_features()

## Search created features

While the `list_features` method allows you to easily view all features of a single
entity type, the [search](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/feature.py#L352) method in the Feature class searches across all featurestores and entity types in a given location (such as `us-central1`), and returns a list of features. This can help you discover features that were created by someone else.

You can query based on feature properties including feature ID, entity type ID, and feature description. You can also limit results by filtering on a specific featurestore, feature value type, and/or labels. Some search examples are shown below. 

Search for all features within a featurestore with the code snippet below.

In [ ]:
my_features = Feature.search(query="featurestore_id={}".format(FEATURESTORE_ID))
my_features

In [ ]:
double_features = Feature.search(
    query="value_type=DOUBLE AND featurestore_id={}".format(FEATURESTORE_ID)
)
double_features[0].gca_resource

Or, limit the search results to features with specific keywords in their ID and type.

In [ ]:
title_features = Feature.search(
    query="feature_id:avg_fare AND value_type=DOUBLE AND featurestore_id={}".format(
        FEATURESTORE_ID
    )
)
title_features[0].gca_resource

In [ ]:
title_features = Feature.search(
    query="feature_id:_10d_rolling_comp AND value_type=DOUBLE AND featurestore_id={}".format(
        FEATURESTORE_ID
    )
)
title_features[0].gca_resource

# Delete Feature Store

admin_client.delete_featurestore(
    request=featurestore_service_pb2.DeleteFeaturestoreRequest(
        name=admin_client.featurestore_path('mwp-mlops-black-friday-350521', 'us-central1', 'teststore2'),
        force=True,
    )
).result()

In [ ]:
print('test')